In [1]:
a = 1

In [1]:
from google.cloud import storage
import numpy as np
from io import BytesIO
import os
from tqdm import tqdm

In [2]:
client = storage.Client(project="musidict-440901")

In [3]:
bucket_name = 'musidict_raw_data'
bucket = client.get_bucket(bucket_name)

In [5]:
blobs = bucket.list_blobs()

In [12]:
npz_bytes = blob.download_as_bytes()

# Load the npz file into a dictionary
data_dict = {}
with np.load(BytesIO(npz_bytes), allow_pickle=True) as data:
    for key in data.files:
        data_dict[key] = data[f"{key}.npy"]

In [4]:
local_directory = 'raw_data'

In [10]:
for blob in tqdm(bucket.list_blobs()):
    if blob.name.endswith('.npz'):
        local_path = os.path.join(local_directory, blob.name.split('/')[-1])
        blob.download_to_filename(local_path)

7139it [27:57,  4.26it/s]


In [7]:
from zipfile import BadZipFile

try:
    np.load("musidict/data/raw_data/114397.npz")
except (EOFError, OSError, ValueError, BadZipFile) as e:
    print("handle")

handle


In [4]:
os.path.getsize("musidict/data/raw_data/114559.npz")

0

In [15]:
import sys
sys.path.append("/home/jparekh4/musidict/")

In [16]:
from src import utils

In [59]:
dataset = []
count = 0
for filename in os.listdir("musidict/data/raw_data/")[:2]:
    data_point = []
    if filename.endswith('.npz') and os.path.getsize(os.path.join("musidict/data/raw_data/", filename)):
        try:
            file_path = os.path.join("musidict/data/raw_data/", filename)
            data = np.load(file_path, allow_pickle=True)
            if "metadata" not in data.keys():
                print(f"{filename} skipped.")
                continue
            genre = utils.preprocess_genres(data["metadata"][0]["genres"])
            print(genre)
            if not genre:
                print(f"{filename} skipped.")
                continue
            data_point.append(genre)
            data_point.append(data["metadata"][0]["bit_rate"])
            data_point.append(data["metadata"][0]["duration"])
            data_point.append(utils.categorize_listens(data["metadata"][0]["listens"]))
            

            data = utils.reshape_all_time_series_data(data)
            data_point.append(np.array(data["mel_spectrogram"]))
            data_point.append(np.array(data["mfccs"]))
            data_point.append(np.array(data["chroma"]))
            data_point.append(np.array(data["spectral_contrast"]))
            data_point.append(np.array(data["zcr"]))
            data_point.append(np.array(data["spectral_centroid"]))
            data_point.append(np.array(data["spectral_bandwidth"]))
            data_point.append(np.array(data["rms_energy"]))
            data_point.append(np.array(data["tonnetz"]))
            print(len(data_point))
            dataset.append(data_point)
            count += 1
            print(f"Total Files Processed: {count}")
        except (EOFError, OSError, ValueError, BadZipFile) as e:
            print(f"{filename} skipped.")
            # raise CustomException(e, sys)
dataset = np.array(dataset, dtype=object)
dataset_df = pd.DataFrame(dataset, columns=[
    "genre",
    "bit_rate",
    "duration",
    "success",
    "mel_spectrogram",
    "mfccs",
    "chroma",
    "spectral_contrast",
    "zcr",
    "spectral_centroid",
    "spectral_bandwidth",
    "rms_energy",
    "tonnetz",
    ])

21
13
Total Files Processed: 1
1235
13
Total Files Processed: 2


In [ ]:
datset_df

In [52]:
import pandas as pd

In [53]:
pd.DataFrame(dataset)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,21,320000,170,hit,"[[62.134373, 93.55765, 160.65472, 196.91096, 2...","[[24.444313, 45.95047, 9.749279, -23.901106, -...","[[0.42347562, 0.120253034, 0.22251274, 0.35785...","[[10.50639746851914, 15.647106600337029, 16.42...","[[0.0654296875, 0.11474609375, 0.15185546875, ...","[[2226.1667734748994, 2373.375721150561, 2716....","[[2187.6527291960438, 2264.697912209358, 2424....","[[0.20811851, 0.22350323, 0.23468891, 0.193144...","[[-0.03982076630060101, -0.03213364967123078, ..."
1,1235,192000,229,average,"[[0.019699413, 0.031691633, 0.0037983505, 3.33...","[[-351.48508, -268.2251, -264.4241, -283.9995,...","[[0.39107373, 0.35774177, 0.33929333, 0.354042...","[[18.04133315461279, 26.87962298732883, 32.650...","[[0.0576171875, 0.06396484375, 0.072265625, 0....","[[670.8332641092563, 676.3803317771357, 664.77...","[[1284.762475296864, 1271.4750849031104, 1262....","[[0.04844471, 0.06618443, 0.07764267, 0.083970...","[[0.07232490624633492, 0.13012848240715416, 0...."


In [55]:
dataset_df = pd.DataFrame(dataset, columns=[
    "genre",
    "bit_rate",
    "duration",
    "success",
    "mel_spectrogram",
    "mfccs",
    "chroma",
    "spectral_contrast",
    "zcr",
    "spectral_centroid",
    "spectral_bandwidth",
    "rms_energy",
    "tonnetz",
    ])

In [56]:
dataset_df

,genre,bit_rate,duration,success,mel_spectrogram,mfccs,chroma,spectral_contrast,zcr,spectral_centroid,spectral_bandwidth,rms_energy,tonnetz
0,21,320000,170,hit,"[[62.134373, 93.55765, 160.65472, 196.91096, 2...","[[24.444313, 45.95047, 9.749279, -23.901106, -...","[[0.42347562, 0.120253034, 0.22251274, 0.35785...","[[10.50639746851914, 15.647106600337029, 16.42...","[[0.0654296875, 0.11474609375, 0.15185546875, ...","[[2226.1667734748994, 2373.375721150561, 2716....","[[2187.6527291960438, 2264.697912209358, 2424....","[[0.20811851, 0.22350323, 0.23468891, 0.193144...","[[-0.03982076630060101, -0.03213364967123078, ..."
1,1235,192000,229,average,"[[0.019699413, 0.031691633, 0.0037983505, 3.33...","[[-351.48508, -268.2251, -264.4241, -283.9995,...","[[0.39107373, 0.35774177, 0.33929333, 0.354042...","[[18.04133315461279, 26.87962298732883, 32.650...","[[0.0576171875, 0.06396484375, 0.072265625, 0....","[[670.8332641092563, 676.3803317771357, 664.77...","[[1284.762475296864, 1271.4750849031104, 1262....","[[0.04844471, 0.06618443, 0.07764267, 0.083970...","[[0.07232490624633492, 0.13012848240715416, 0...."


In [61]:
t_dash = np.load("musidict/data/transformed_data/transformed_dataset.npy", allow_pickle=True)

In [66]:
t_dash[:, 4]

(4083,)

In [67]:
import torch

In [73]:
torch.tensor(np.stack(t_dash[:, 3]))

ValueError: all input arrays must have the same shape

In [78]:
for i in range(4083):
    if t_dash[i, 3].shape != (128, 937):
        print(i)

242
2336
2705


In [81]:
t_dash[2705, 3].shape

(128, 51)

In [82]:
t_dash[2336, 3].shape

(128, 49)

In [83]:
def normalize_array_shape(array, target_shape=(128, 937)):
    # Truncate if the array is larger than the target shape
    truncated_array = array[:target_shape[0], :target_shape[1]]
    
    # Pad if the array is smaller than the target shape
    padding = ((0, max(0, target_shape[0] - truncated_array.shape[0])),
               (0, max(0, target_shape[1] - truncated_array.shape[1])))
    
    normalized_array = np.pad(truncated_array, padding, mode='constant')
    return normalized_array

In [85]:
res = normalize_array_shape(t_dash[2336, 3])

In [86]:
res.shape

(128, 937)